## 图像识别入门工具包

11/5/2020 承轩

### 下载数据

In [ ]:
!apt install aria2


In [3]:
# 请替换成你的下载地址 然后耐心等候
!aria2c -x10 "http://aidownload.futurelab.tv/af2020cv-2020-05-09-v5-dev.zip?e=1589276753&token=kO26gOFamzTLWaBGhBdGdua2WO4ejK-xeVnrFJMk:knUySS-5BRcbQ_jB3xDct6GPyYU=&sign=7467056f43a9e8532456bb1c43d68cca&t=5eb97331"


07/19 08:26:37 [NOTICE] Downloading 1 item(s)

07/19 08:26:39 [ERROR] CUID#7 - Download aborted. URI=http://aidownload.futurelab.tv/af2020cv-2020-05-09-v5-dev.zip?e=1589276753&token=kO26gOFamzTLWaBGhBdGdua2WO4ejK-xeVnrFJMk:knUySS-5BRcbQ_jB3xDct6GPyYU=&sign=7467056f43a9e8532456bb1c43d68cca&t=5eb97331
Exception: [AbstractCommand.cc:351] errorCode=24 URI=http://aidownload.futurelab.tv/af2020cv-2020-05-09-v5-dev.zip?e=1589276753&token=kO26gOFamzTLWaBGhBdGdua2WO4ejK-xeVnrFJMk:knUySS-5BRcbQ_jB3xDct6GPyYU=&sign=7467056f43a9e8532456bb1c43d68cca&t=5eb97331
  -> [HttpSkipResponseCommand.cc:215] errorCode=24 Authorization failed.

07/19 08:26:39 [NOTICE] Download GID#28faa2249546e3f9 not complete: 

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
28faa2|ERR |       0B/s|http://aidownload.futurelab.tv/af2020cv-2020-05-09-v5-dev.zip?e=1589276753&token=kO26gOFamzTLWaBGhBdGdua2WO4ejK-xeVnrFJMk:knUySS-5BRcbQ_jB3xDct6GP

In [ ]:
# 耐心等待5分钟
!unzip -q -P Platax af2020cv-2020-05-09-v5-dev.zip

In [4]:
!pip install skorch

     |████████████████████████████████| 122kB 7.3MB/s 


In [5]:
import numpy as np
import pandas as pd
from torch import nn
import torch.nn.functional as F
import torch

from skorch import NeuralNetClassifier
from torch.utils.data import Dataset
from PIL import Image
from torchvision import datasets, models, transforms
import torchvision.transforms as transforms
from skorch.helper import predefined_split

### Dataloader

In [6]:
def mixup_data(x, y, alpha=1.0, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    #if use_cuda:
      #x, y = x.cuda(), y.cuda()
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    '''if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)'''
    index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

class Train(Dataset):
  def __init__(self):
    super().__init__()
    self.datas = pd.read_csv('/content/af2020cv-2020-05-09-v5-dev/training.csv')
    self.trans = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
  def __len__(self):
    return len(self.datas)
  def pil_loader(self,path):
      with open(path, 'rb') as f:
          img = Image.open(f)
          return img.convert('RGB')
  def __getitem__(self, index):
    row = self.datas.iloc[index]
    fname = '/content/af2020cv-2020-05-09-v5-dev/data/{}.jpg'.format(row['FileID'])
    label = row['SpeciesID']
    img = self.pil_loader(fname)
    img = self.trans(img)
    inputs, target_a, targets_b, lam = mixup_data(img, label)
    return inputs, target_a, target_b

class Val(Dataset):
  def __init__(self):
    super().__init__()
    self.datas = pd.read_csv('/content/af2020cv-2020-05-09-v5-dev/annotation.csv')
    self.trans = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
  def __len__(self):
    return len(self.datas)
  def pil_loader(self,path):
      with open(path, 'rb') as f:
          img = Image.open(f)
          return img.convert('RGB')
  def __getitem__(self, index):
    row = self.datas.iloc[index]
    fname = '/content/af2020cv-2020-05-09-v5-dev/data/{}.jpg'.format(row['FileID'])
    label = row['SpeciesID']
    img = self.pil_loader(fname)
    img = self.trans(img)
    return img, int(label)

tra = Train()
print(len(tra))
print(tra.__getitem__(1))
val = Val()
print(len(val))

FileNotFoundError: ignored

### 写你的Model并训练

In [1]:
from skorch.callbacks import LRScheduler
from torch.optim.lr_scheduler import CosineAnnealingLR

from torch.autograd import Variable
class FixLoss(nn.Module):
  def __init__(self, class_num=20):
    super(FixLoss, self).__init__()
    self.criterion = nn.CrossEntropyLoss()

  def forward(pred, y_a, y_b, lam):
    return lam * self.criterion(pred, y_a) + (1 - lam) * self.criterion(pred, y_b)


class FocalLoss(nn.Module):

  def __init__(self, class_num=20, alpha=None, gamma=2, size_average=True):
    super(FocalLoss, self).__init__()
    if alpha is None:
      self.alpha = Variable(torch.ones(class_num, 1))
    else:
      if isinstance(alpha, Variable):
        self.alpha = alpha
      else:
        self.alpha = Variable(alpha)
    self.gamma = gamma
    self.class_num = class_num
    self.size_average = size_average

  def forward(self, inputs, targets):
    N = inputs.size(0)
    C = inputs.size(1)
    P = F.softmax(inputs)

    class_mask = inputs.data.new(N, C).fill_(0)
    class_mask = Variable(class_mask)
    ids = targets.view(-1, 1)
    class_mask.scatter_(1, ids.data, 1.)
    #print(class_mask)


    if inputs.is_cuda and not self.alpha.is_cuda:
      self.alpha = self.alpha.cuda()
    alpha = self.alpha[ids.data.view(-1)]

    probs = (P*class_mask).sum(1).view(-1,1)

    log_p = probs.log()
    #print('probs size= {}'.format(probs.size()))
    #print(probs)

    batch_loss = -alpha*(torch.pow((1-probs), self.gamma))*log_p 
    #print('-----bacth_loss------')
    #print(batch_loss)


    if self.size_average:
      loss = batch_loss.mean()
    else:
      loss = batch_loss.sum()
    return loss

class MyModule(nn.Module):
  def __init__(self, num_classes=20):
    super(MyModule, self).__init__()

    model_ft = models.densenet161(pretrained=True)
    num_ftrs = model_ft.classifier.in_features
    model_ft.classifier = nn.Linear(num_ftrs, num_classes)
    self.model_ft = model_ft

  def forward(self, X, **kwargs):
    X = self.model_ft(X)
    return X

# 测试时增强、使用动量SGD替代ADAM、使用更好的预训练网络、从后往前逐步解冻预训练网络、对训练数据进行增强、调整BS和LR等
lambda1 = lambda epoch:epoch//30
lambda2 = lambda epoch:0.95**epoch
lambda3 = lambda epoch:0.95
net = NeuralNetClassifier(
    MyModule,
    criterion=FixLoss,
    max_epochs=100,
    lr=0.01,  # LR
    batch_size=32,  # BS
    optimizer=torch.optim.SGD,  # using SGD
    #optimizer__momentum=0.1,
    callbacks=[
      ('lr_Scheduler',LRScheduler(policy=CosineAnnealingLR, T_max=10, eta_min=4e-8, last_epoch=-1))
    ],
    train_split=predefined_split(val),
    iterator_train__shuffle=True,
    #device='cuda',
    iterator_train__num_workers=32,
    iterator_train__pin_memory=True,
)
net.fit(tra, y=None)

ModuleNotFoundError: ignored